In [355]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [356]:
pd.options.display.max_columns = 100

In [357]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [358]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [359]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [360]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [361]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [362]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                                              Titre  \
0    3595                                     Whity (vostfr)   
1    3594           Tous les autres s'appellent Ali (vostfr)   
2    3593                                  Querelle (vostfr)   
3    3592  Pourquoi monsieur R. est-il atteint de folie m...   
4    3591                    Pionniers à Ingolstadt (vostfr)   

                             Titre alternatif Année  Durée  \
0                                         NaN  1971     95   
1      Angst essen Seele auf (Titre original)  1974     93   
2                                         NaN  1982    108   
3  Warum läuft Herr R. Amok? (Titre original)  1970     84   
4     Pioniere in Ingolstadt (Titre original)  1971     84   

  Chemin de l'affiche                        Bande-annonce  \
0          003679.jpg  www.youtube.com/watch?v=Ni1bwcRNndY   
1          003678.jpg  www.youtube.com/watch?v=LsLlXnmxWtM   
2          003677.jpg  www.youtube.com/watch?v=kjc6I_0PP

In [363]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [364]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [365]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [366]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [367]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [368]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [369]:
display(df_director)

,id,Réalisateur(s)
0,1,Rainer Werner Fassbinder
1,2,Michael Fengler
2,3,Volker Schlöndorff
3,4,Yasuo Furuhata
4,5,Norifumi Suzuki
...,...,...
1759,1760,Richard C. Sarafian
1760,1761,Chris Petit
1761,1762,Brett Ratner
1762,1763,Buster Keaton


In [370]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,003679.jpg,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,003678.jpg,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,003677.jpg,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,003676.jpg,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,003675.jpg,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [371]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3595
1,0,3594
2,0,3593
3,0,3592
4,0,3591
...,...,...
3868,1759,20
3869,1760,18
3870,1761,15
3871,1762,5


In [372]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id            Réalisateur(s)
0  1765  Rainer Werner Fassbinder
1     2           Michael Fengler
2     3        Volker Schlöndorff
3     4            Yasuo Furuhata
4     5           Norifumi Suzuki
     id  Numéro
0  1765    3595
1  1765    3594
2  1765    3593
3  1765    3592
4  1765    3591


3873

In [373]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [374]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [375]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3595

# table movie, screenwriter et intermediaire

In [376]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [377]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [378]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [379]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [380]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [381]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [382]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Rainer Werner Fassbinder
1,1,NaN
2,2,Burkhard Driest
3,3,Jean Genet
4,4,Michael Fengler
...,...,...
3860,3860,Ted Tally
3861,3861,David Veloz
3862,3862,Richard Rutowski
3863,3863,Buster Keaton


In [383]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [384]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3595
1,0,3593
2,0,3592
3,0,3591
4,0,3590
...,...,...
6964,3860,15
6965,3861,14
6966,3862,14
6967,3863,5


In [385]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id             Scénariste(s)
0  3865  Rainer Werner Fassbinder
1     0                       NaN
2     2           Burkhard Driest
3     3                Jean Genet
4     4           Michael Fengler
     id  Numéro
0  3865    3595
1  3865    3593
2  3865    3592
3  3865    3591
4  3865    3590


6969

In [386]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [387]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [388]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [389]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [390]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [391]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [392]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [393]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [394]:
display(df_music)

,id,Compositeur(s)
0,0,Peer Raben
1,1,Rainer Werner Fassbinder
2,2,NaN
3,3,Holger Münzer
4,4,Klaus Doldinger
...,...,...
1265,1265,Machine Head
1266,1266,Photek
1267,1267,John Frizzell
1268,1268,Leonard Cohen


In [395]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [396]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3595
1,0,3593
2,0,3592
3,0,3591
4,0,3590
...,...,...
3980,1265,46
3981,1266,46
3982,1267,42
3983,1268,14


In [397]:
# # Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
# null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# # Si une ligne avec 'Compositeur(s)' NULL est trouvée
# if not null_music_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_music.loc[null_music_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_music['id'].max()
#     if max_id > 0:
#         df_music.loc[df_music.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_music.head())

# # Enregistrer le DataFrame modifié dans la table 'music' de la base de données
# df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_music.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


In [398]:
df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

1270

In [399]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

3985

# table movie, studio et intermediaire

In [400]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [401]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [402]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [403]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [404]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [405]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [406]:
display(df_studio)

,id,Studio
0,0,Atlantis-Film
1,1,Filmverlag der Autoren
2,2,Gaumont
3,3,Süddeutscher Rundfunk
4,4,Antiteater-X-Film
...,...,...
942,942,A-Film Distribution
943,943,TFC
944,944,Dybex
945,945,British Empire Films Australia


In [407]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [408]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3595
1,1,3594
2,1,3588
3,1,3585
4,2,3593
...,...,...
3592,942,172
3593,943,171
3594,944,169
3595,945,108


In [409]:
# # Trouver l'index de la ligne avec le 'Studio' NULL
# null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# # Si une ligne avec 'Studio' NULL est trouvée
# if not null_studio_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_studio.loc[null_studio_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_studio['id'].max()
#     if max_id > 0:
#         df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_studio.head())

# # Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
# df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_studio.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


In [410]:
df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

947

In [411]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3597

# table movie, casting & intermediaire

In [412]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [413]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [414]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [415]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [416]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [417]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [418]:
display(df_casting)

,id,Distribution
0,0,Günther Kaufmann
1,1,Ulli Lommel
2,2,Harry Baer
3,3,Hanna Schygulla
4,4,Ron Randell
...,...,...
9842,9842,Paulette Goddard
9843,9843,Henry Bergman
9844,9844,Tiny Sandford
9845,9845,Jackie Coogan


In [419]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [420]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3595
1,0,3567
2,1,3595
3,1,3582
4,1,3575
...,...,...
17060,9842,2
17061,9843,2
17062,9844,2
17063,9845,1


In [421]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


     id      Distribution
0  9847  Günther Kaufmann
1     1       Ulli Lommel
2     2        Harry Baer
3     3   Hanna Schygulla
4     4       Ron Randell
     id  Numéro
0  9847    3595
1  9847    3567
2     1    3595
3     1    3582
4     1    3575


17065

In [422]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [423]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [424]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [425]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [426]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [427]:
df_country_movie  = df_country_movie.explode("Pays")

In [428]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [429]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [430]:
display(df_country)

,id,Pays
0,0,Allemagne
1,1,France
2,2,Japon
3,3,Etats-Unis
4,4,Danemark
5,5,IE
6,6,Belgique
7,7,Italie
8,8,Royaume-Uni
9,9,Suède


In [431]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [432]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3595
1,0,3594
2,0,3592
3,0,3591
4,0,3590
...,...,...
3590,42,853
3591,43,921
3592,44,769
3593,45,631


In [433]:
# # Trouver l'index de la ligne avec le 'Pays' NULL
# null_country_index = df_country[df_country['Pays'].isnull()].index

# # Si une ligne avec 'Pays' NULL est trouvée
# if not null_country_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_country.loc[null_country_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_country['id'].max()
#     if max_id > 0:
#         df_country.loc[df_country.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_country.head())

# # Enregistrer le DataFrame modifié dans la table 'country' de la base de données
# df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_country.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

In [434]:
df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

46

In [435]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3595

# table movie, genre & intermediaire

In [436]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [437]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [438]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [439]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [440]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [441]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [442]:
display(df_genre)

,id,Genre(s)
0,0,Western
1,1,Romantique
2,2,Drame
3,3,Comédie
4,4,Suspense
5,5,Action
6,6,Mystère
7,7,Policier
8,8,Histoire
9,9,Aventures


In [443]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [444]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3595
1,0,3555
2,0,3520
3,0,3518
4,0,3276
...,...,...
8037,38,920
8038,38,642
8039,38,384
8040,38,352


In [445]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id    Genre(s)
0  39     Western
1   1  Romantique
2   2       Drame
3   3     Comédie
4   4    Suspense
   id  Numéro
0  39    3595
1  39    3555
2  39    3520
3  39    3518
4  39    3276


8042

In [446]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [447]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

8042

# table movie, language & intermediaire

In [448]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [449]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [450]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [451]:
df_language_movie  = df_language_movie.explode("Langues")

In [452]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [453]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [454]:
display(df_language)

,id,Langues
0,0,Allemand
1,1,Anglais
2,2,Arabe
3,3,Français
4,4,Espagnol
5,5,Japonais
6,6,Grec
7,7,Italien
8,8,Russe
9,9,Danois


In [455]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [456]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3595
1,0,3594
2,0,3593
3,0,3592
4,0,3591
...,...,...
5068,42,1060
5069,43,1053
5070,43,930
5071,43,926


In [457]:
# # Trouver l'index de la ligne avec les 'Langues' NULL
# null_language_index = df_language[df_language['Langues'].isnull()].index

# # Si une ligne avec 'Langues' NULL est trouvée
# if not null_language_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_language.loc[null_language_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_language['id'].max()
#     if max_id > 0:
#         df_language.loc[df_language.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_language.head())

# # Enregistrer le DataFrame modifié dans la table 'language' de la base de données
# df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_language.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


In [458]:
df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

45

In [459]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5073

# table, movie, tag & intermediaire

In [460]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [461]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [462]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [463]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [464]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [465]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [466]:
display(df_tag)

,id,Etiquettes
0,0,NaN
1,1,Ours d'Or
2,2,Meta-Film
3,3,Survival
4,4,Marvel
...,...,...
201,201,Momie
202,202,viking
203,203,Réalisme soviétique
204,204,Guy Debord


In [467]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3595,Whity (vostfr),NaN,1971,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Ni1bwcRNndY,NaN,"En 1878, quelque part dans l'Ouest américain, ...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Whity...,Matroska (MKV),NaN,Fichier multimédia,1.22 GB,movie/105757,67979.0
1,3594,Tous les autres s'appellent Ali (vostfr),Angst essen Seele auf (Titre original),1974,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=LsLlXnmxWtM,NaN,Dans un café fréquenté par des travailleurs im...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Tous ...,Matroska (MKV),NaN,Fichier multimédia,1.70 GB,movie/216,71141.0
2,3593,Querelle (vostfr),NaN,1982,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kjc6I_0PP6c,It will take you into a surreal world of passi...,Le Vengeur vient d'accoster à Brest. Sur le po...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Quere...,Matroska (MKV),NaN,Fichier multimédia,1.31 GB,movie/42135,84565.0
3,3592,Pourquoi monsieur R. est-il atteint de folie m...,Warum läuft Herr R. Amok? (Titre original),1970,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"Un père de famille, dessinateur industriel, mè...",[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pourq...,Matroska (MKV),NaN,Fichier multimédia,773 MB,movie/10308,66546.0
4,3591,Pionniers à Ingolstadt (vostfr),Pioniere in Ingolstadt (Titre original),1971,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Des ingénieurs de l'armée sont envoyés dans un...,[My Book]F:\#DIVX\DIVX MOVIEZ\fassbinder\Pioni...,Matroska (MKV),NaN,Fichier multimédia,1.23 GB,movie/191703,67584.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3591,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3592,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3593,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [468]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3595
1,0,3594
2,0,3593
3,0,3592
4,0,3591
...,...,...
4543,204,74
4544,204,73
4545,205,17
4546,205,16


In [469]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id Etiquettes
0  206        NaN
1    1  Ours d'Or
2    2  Meta-Film
3    3   Survival
4    4     Marvel
    id  Numéro
0  206    3595
1  206    3594
2  206    3593
3  206    3592
4  206    3591


4548

In [470]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [471]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )